In [27]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [28]:
import h2o
import mpld3
mpld3.enable_notebook()
import matplotlib.pyplot as plt
from src import measure_repository
from src import repository_adapter
from matplotlib.dates import  DateFormatter
from matplotlib.dates import YearLocator
from scipy.ndimage import gaussian_filter
import numpy as np

h2o.init(ip="local.h2o.air_data", port=54321)

Checking whether there is an H2O instance running at http://local.h2o.air_data:54321. connected.


H2O cluster uptime:,2 days 2 hours 14 mins
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.1
H2O cluster version age:,2 months and 17 days
H2O cluster name:,root
H2O cluster total nodes:,1
H2O cluster free memory:,10.27 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [39]:
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit


def validate(metering_point_id):
    
    
    initial_dataset = repository_adapter.run_sql(
        """ SELECT
                am.result as value,
                #date,
                HOUR(am.date) as hour,
                MONTH(am.date) as month,
                WEEKDAY(am.date)+1 as weekday,
                WEEK(am.date) as week
            FROM air.air_quality_measures am
            INNER JOIN air.weather_measures wm
                ON am.date = wm.time
            WHERE 
                am.metering_point_id = %s
                AND am.result IS NOT NULL
                AND YEAR(am.date) IN (2015, 2016, 2017)
                #AND MONTH(am.date) IN (1)
            ORDER BY am.date ASC
            ;""" % (metering_point_id),
        [
            "value", 
            "hour", 
            "month", 
            "weekday", 
            "week",
        ]
    )

    x = initial_dataset.iloc[:, 1:]
    y = initial_dataset.iloc[:, [0]]

    cross_validate(
        x,
        y,
        12,
        lambda x, y, train_dataset, validation_dataset: train_with_plots(x, y, train_dataset, validation_dataset)
    )
    
def cross_validate(X: pd.DataFrame, y: pd.DataFrame, splits: int, callback):

    tscv = TimeSeriesSplit(n_splits=splits)
    for train_index, test_index in tscv.split(X):
        X_train = pd.DataFrame(X.values[train_index], columns=X.columns.values)
        X_test = pd.DataFrame(X.values[test_index], columns=X.columns.values)
        y_train = pd.DataFrame(y.values[train_index], columns=y.columns.values)
        y_test = pd.DataFrame(y.values[test_index], columns=y.columns.values)

        train_dataset = y_train.join(X_train)
        validation_dataset = y_test.join(X_test)

        callback(X, y, train_dataset, validation_dataset)
        
        
def train_with_plots(x, y, train_dataset, validation_dataset):

    training_frame = h2o.H2OFrame(training_dataset.values)
    training_frame.set_names(training_dataset.columns.values.tolist())
    validation_frame = h2o.H2OFrame(validation_dataset.values)
    validation_frame.set_names(validation_dataset.columns.values.tolist())

    columns = training_frame.columns
    columns.pop(0)
    X = columns
    Y = 'value'


    model = H2OXGBoostEstimator(
        ntrees=200
         , max_depth=10
        , learn_rate=0.01
        , sample_rate=0.8
        , backend="cpu"
    )


    model.train(x=X, y=Y, training_frame=training_frame, validation_frame=validation_frame)
    y_pred = model.predict(validation_frame)

    y_test = validation_dataset.iloc[:, 0].values
    x_test = validation_dataset.iloc[:, 1].values

    plt.figure(figsize=(13, 6))
    step = (float(y_test.max()) - float(y_test.min())) / 10.0
    plt.yticks(np.arange(float(y_test.min()), float(y_test.max()), step))
    plt.plot(y_test, color='red')
    plt.plot(y_pred.as_data_frame().values, color='green')
#     plt.plot(y, color='blue')
    plt.show()


# validate(2)
